## Hybrid Algorithm (3-frame differencing + Adaptive background modeling)

In attempt to join the qualities of motion detection algorithms a hybrid algorithm was proposed,
which is a modification of 3-frame difference with background adaptation problems. The implemented
algorithm is presented in the figure below. The output with analysis against different conditions will help us
classify the optimum algorithm for our system.


<img src="etc/hybrid.png" />

In [ ]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture(0)

alpha = 0.5 # initialization

time.sleep(3)

_, first_frame = cap.read()
first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

_, second_frame = cap.read()
second_gray = cv2.cvtColor(second_frame, cv2.COLOR_BGR2GRAY)

Tn = 100 * np.ones(first_gray.shape, np.uint8)


def bck_model(Bn, In, dilute_frame, alpha):
    '''
        Implementing background adaptation based on model formulae
    '''
    
    B_new = Bn
    row = Bn.shape[0]
    col = Bn.shape[1]
    
    for i in range(0, row):
        for j in range(0, col):
            if(dilute_frame[i][j] > 0): # moving pixel
                B_new[i][j] = (alpha * Bn[i][j] + (1 - alpha)* In[i][j]).astype(np.uint8)
                
    return B_new


def thresh_model(Tn, Bn, In, dilute_frame, alpha):
    ''' 
        Updating the threshold matrix
    '''

    T_new = Tn
    row = Tn.shape[0]
    col = Tn.shape[1]
    
    for i in range(0, row):
        for j in range(0, col):
            if(dilute_frame[i][j] == 1): # moving pixel
                T_new[i][j] = (alpha * Tn[i][j] + (1 - alpha)* 5 * abs(In[i][j] - Bn[i][j]) ).astype(np.uint8)
                
    return T_new


def seg_motion(delta, Tn):
    ''' 
        Compare frame difference matrix with threshold matrix for motion 
    '''
    row = delta.shape[0]
    col = delta.shape[1]
    thresh_mat = np.zeros([row, col])
    for i in range(0, row):
        for j in range(0, col):
            if(delta[i][j] >= Tn[i][j]): # Binary Thresholding
                thresh_mat[i][j] = 1
                
    return thresh_mat


while(True):
    
    _, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21,21), 0)
    
    delta_frame1 = cv2.absdiff(gray, first_gray)
    delta_frame2 = cv2.absdiff(gray, second_gray)
    
    thresh_frame1 = seg_motion(delta_frame1, Tn)
    thresh_frame2 = seg_motion(delta_frame2, Tn)
    
    thresh_frame = cv2.bitwise_and(thresh_frame1, thresh_frame2)
    
    dilute_frame = cv2.dilate(thresh_frame, None, iterations = 3)
    
    # cv2.imshow("gray", gray)
    cv2.imshow("motion", dilute_frame)
    cv2.imshow("Original", frame)
    
    second_gray = first_gray
    # updating frame
    first_gray = bck_model( first_gray, gray, dilute_frame, alpha)
    # update threshold matrix
    Tn = thresh_model(Tn, first_gray, gray, dilute_frame, alpha)
    
    if cv2.waitKey(10) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()
    

/home/nikhil/.virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in ubyte_scalars
